Setup

In [2]:
!cat ~/.keras/keras.json

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

Rank-5 Accuracy

In [3]:
# import the necessary packages
import numpy as np

def rank5_accuracy(preds, labels):
	# initialize the rank-1 and rank-5 accuracies
	rank1 = 0
	rank5 = 0

	# loop over the predictions and ground-truth labels
	for (p, gt) in zip(preds, labels):
		# sort the probabilities by their index in descending
		# order so that the more confident guesses are at the
		# front of the list
		p = np.argsort(p)[::-1]

		# check if the ground-truth label is in the top-5
		# predictions
		if gt in p[:5]:
			rank5 += 1

		# check to see if the ground-truth is the #1 prediction
		if gt == p[0]:
			rank1 += 1

	# compute the final rank-1 and rank-5 accuracies
	rank1 /= float(len(preds))
	rank5 /= float(len(preds))

	# return a tuple of the rank-1 and rank-5 accuracies
	return (rank1, rank5)

Rank Accuracy

In [4]:
import pickle
import h5py

In [5]:
def rank_accuracy(db_filename, model_filename):
    # load the pre-trained model
    print("[INFO] loading pre-trained model...")
    model = pickle.loads(open(model_filename, "rb").read())

    # open the HDF5 database for reading then determine the index of
    # the training and testing split, provided that this data was
    # already shuffled *prior* to writing it to disk
    db = h5py.File(db_filename, "r")
    i = int(db["labels"].shape[0] * 0.75)

    # make predictions on the testing set then compute the rank-1
    # and rank-5 accuracies
    print("[INFO] predicting...")
    preds = model.predict_proba(db["features"][i:])
    (rank1, rank5) = rank5_accuracy(preds, db["labels"][i:])

    # display the rank-1 and rank-5 accuracies
    print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
    print("[INFO] rank-5: {:.2f}%".format(rank5 * 100))

    # close the database
    db.close() 

In [6]:
rank_accuracy(db_filename="drive/MyDrive/pyimagesearch/output/21-feature-extraction/animals_features.hdf5", model_filename="drive/MyDrive/pyimagesearch/output/21-feature-extraction/animals.cpickle")

[INFO] loading pre-trained model...
[INFO] predicting...
[INFO] rank-1: 98.40%
[INFO] rank-5: 100.00%


In [6]:
rank_accuracy(db_filename="drive/MyDrive/pyimagesearch/output/21-feature-extraction/flowers17_features.hdf5", model_filename="drive/MyDrive/pyimagesearch/output/21-feature-extraction/flowers17.cpickle")

[INFO] loading pre-trained model...
[INFO] predicting...
[INFO] rank-1: 90.00%
[INFO] rank-5: 99.12%
